Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [31]:
# Logistic regression model
graph = tf.Graph()

# regularization parameter
beta = 0.001

batch_size = 128

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(
      tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
      beta*tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
      tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [32]:
num_steps = 3001

with tf.Session(graph=graph) as session:  
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, batch_labels))      
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.847260
Training accuracy: 14.1%
Validation accuracy: 18.0%
Loss at step 500: 2.724689
Training accuracy: 76.6%
Validation accuracy: 75.8%
Loss at step 1000: 1.685281
Training accuracy: 78.9%
Validation accuracy: 78.7%
Loss at step 1500: 1.256439
Training accuracy: 82.8%
Validation accuracy: 79.4%
Loss at step 2000: 0.964314
Training accuracy: 83.6%
Validation accuracy: 80.8%
Loss at step 2500: 0.794276
Training accuracy: 79.7%
Validation accuracy: 81.8%
Loss at step 3000: 0.648932
Training accuracy: 83.6%
Validation accuracy: 81.8%
Test accuracy: 88.5%


In [33]:
# 1 hidden-layer neural network

beta1 = 0.001
beta2 = 0.001

graph = tf.Graph()
with graph.as_default():
    # input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(None, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # variables
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    b1 = tf.Variable(tf.zeros([1024]))
    
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)
    
    W2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    # training computation
    logits = tf.matmul(h1, W2) + b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +\
        beta1*tf.nn.l2_loss(W1) + beta2*tf.nn.l2_loss(W2)
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1), W2) + b2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1), W2) + b2)

In [34]:
num_steps = 3001
batch_size = 128

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 600.226196
Minibatch accuracy: 5.5%
Validation accuracy: 26.5%
Minibatch loss at step 500: 193.894241
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 116.043518
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 68.535889
Minibatch accuracy: 85.2%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 41.717186
Minibatch accuracy: 82.0%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 25.299358
Minibatch accuracy: 87.5%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 15.448955
Minibatch accuracy: 88.3%
Validation accuracy: 87.2%
Test accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [36]:
num_steps = 3001
batch_size = 128

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0]/1000 - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 665.569885
Minibatch accuracy: 11.7%
Validation accuracy: 33.9%
Minibatch loss at step 500: 190.853119
Minibatch accuracy: 100.0%
Validation accuracy: 69.4%
Minibatch loss at step 1000: 115.743820
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Minibatch loss at step 1500: 70.193413
Minibatch accuracy: 100.0%
Validation accuracy: 69.4%
Minibatch loss at step 2000: 42.569111
Minibatch accuracy: 100.0%
Validation accuracy: 69.4%
Minibatch loss at step 2500: 25.816265
Minibatch accuracy: 100.0%
Validation accuracy: 69.3%
Minibatch loss at step 3000: 15.656461
Minibatch accuracy: 100.0%
Validation accuracy: 69.4%
Test accuracy: 76.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [73]:
# 1 hidden-layer neural network

beta1 = 0.001
beta2 = 0.001
dropout_rate = 0.5

graph = tf.Graph()
with graph.as_default():
    # input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(None, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # variables
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    b1 = tf.Variable(tf.zeros([1024]))
    
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)
    
    keep_prob1 = tf.placeholder(tf.float32)
    h1_drop = tf.nn.dropout(h1, keep_prob1)
    
    W2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    # training computation
    logits = tf.matmul(h1_drop, W2) + b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +\
        beta1*tf.nn.l2_loss(W1) + beta2*tf.nn.l2_loss(W2)
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1), W2) + b2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1), W2) + b2)

In [74]:
num_steps = 3001
batch_size = 128

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,\
                    keep_prob1: dropout_rate}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 780.089966
Minibatch accuracy: 11.7%
Validation accuracy: 22.4%
Minibatch loss at step 500: 213.095078
Minibatch accuracy: 75.8%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 117.117729
Minibatch accuracy: 74.2%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 70.004784
Minibatch accuracy: 74.2%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 41.465496
Minibatch accuracy: 82.8%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 25.287481
Minibatch accuracy: 81.2%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 15.426160
Minibatch accuracy: 82.8%
Validation accuracy: 85.8%
Test accuracy: 91.9%


In [72]:
num_steps = 3001
batch_size = 128

dropout_rate_extreme = 0.5

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0]/1000 - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,\
                    keep_prob1: dropout_rate_extreme}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 782.092346
Minibatch accuracy: 13.3%
Validation accuracy: 34.0%
Minibatch loss at step 500: 190.839325
Minibatch accuracy: 100.0%
Validation accuracy: 72.1%
Minibatch loss at step 1000: 115.759933
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 1500: 70.209663
Minibatch accuracy: 100.0%
Validation accuracy: 71.9%
Minibatch loss at step 2000: 42.583393
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 2500: 25.826340
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 3000: 15.662842
Minibatch accuracy: 100.0%
Validation accuracy: 72.5%
Test accuracy: 79.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [170]:
# 2 hidden-layer neural network

beta1 = 0.001
beta2 = 0.001
beta3 = 0.001
dropout_rate = 0.5

n_W1 = 1024
n_W2 = 1024

graph = tf.Graph()
with graph.as_default():
    # input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(None, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # variables
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_W1],\
                    stddev=np.sqrt(2.0/(image_size * image_size))))
    b1 = tf.Variable(tf.zeros([n_W1]))
    
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)
    
    keep_prob1 = tf.placeholder(tf.float32)
    h1_drop = tf.nn.dropout(h1, keep_prob1)
    
    W2 = tf.Variable(tf.truncated_normal([n_W1, n_W2],\
                                        stddev=np.sqrt(2.0/(n_W1))))
    b2 = tf.Variable(tf.zeros([n_W2]))
    
    h2 = tf.nn.relu(tf.matmul(h1_drop, W2) + b2)
    
    keep_prob2 = tf.placeholder(tf.float32)
    h2_drop = tf.nn.dropout(h2, keep_prob2)
    
    W3 = tf.Variable(tf.truncated_normal([n_W2, num_labels],\
                                        stddev=np.sqrt(2.0/(n_W2))))
    b3 = tf.Variable(tf.zeros([num_labels]))
    
    # training computation
    logits = tf.matmul(h2_drop, W3) + b3
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +\
        beta1*tf.nn.l2_loss(W1) + beta2*tf.nn.l2_loss(W2) +\
        beta3*tf.nn.l2_loss(W3)
    
    # optimizer
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.1
    learning_rate = tf.train.exponential_decay(starter_learning_rate,\
                                              global_step, 3000, 0.95,\
                                              staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,\
                                            global_step=global_step)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1), W2) + b2), W3) + b3)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1), W2) + b2), W3) + b3)

In [171]:
num_steps = 9001
batch_size = 128

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,\
                    keep_prob1: dropout_rate, keep_prob2 : dropout_rate}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 4.352854
Minibatch accuracy: 12.5%
Validation accuracy: 25.4%
Minibatch loss at step 1000: 1.825747
Minibatch accuracy: 83.6%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 1.652989
Minibatch accuracy: 86.7%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 1.350352
Minibatch accuracy: 88.3%
Validation accuracy: 87.1%
Minibatch loss at step 4000: 1.156906
Minibatch accuracy: 88.3%
Validation accuracy: 87.5%
Minibatch loss at step 5000: 1.091223
Minibatch accuracy: 86.7%
Validation accuracy: 87.9%
Minibatch loss at step 6000: 0.957455
Minibatch accuracy: 86.7%
Validation accuracy: 88.2%
Minibatch loss at step 7000: 0.981604
Minibatch accuracy: 82.8%
Validation accuracy: 88.6%
Minibatch loss at step 8000: 0.823562
Minibatch accuracy: 86.7%
Validation accuracy: 88.7%
Minibatch loss at step 9000: 0.768183
Minibatch accuracy: 87.5%
Validation accuracy: 88.9%
Test accuracy: 94.4%
